In [25]:
import os
from os.path import expanduser
import socket
import sys
import glob
import time

# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "pyfesom")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"pyfesom")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules") # add custom Vibe 's modules
#sys.path.append("pyfesom") # add standard 's modules
#########################################

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
import pyfesom as pf
#from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import cmocean as cmaps
import pandas as pd
#from matplotlib.colors import ListedColormap

computer =  bfn1001
working directory set to /home/hbkoziel/pyfesom


In [26]:
# ==============================================================================
# Running this file loads tracers from old FESOM-REcoM2 output file (oce.mean.nc)
# and saves each tracer in an individual file
# 
#  Input:
#  - mesh_id: Name of mesh, will be added to the netcdf name
#  - meshpath: Speciefies where the target mesh is stored
#  - save_netcdf: If true, netcdf will be created
#  - delete_old_netcdf: If a netcdf file with the same name exists, a new cannot 
#    be made. If set to true, an old netcdf with the same name will be deleted
#
#  Output:
#  - netcdf file for each tracer in the old file
#  
#  During running, keep an eye on the output in the terminal, to see if it 
#  makes sense. 
#
# ==============================================================================
# Loading mesh for run

mesh_id    = 'meshArc4.5'
get3d = False
meshpath   = wd_path+'/data/mesh/'+mesh_id+'/'            # Defining path where mesh is stored
mesh = pf.load_mesh(meshpath, usepickle=True,get3d=get3d)           # Loading mesh, stores it in mesh.****  
mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

tracername = 'area'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)

# choose RUN
runid   = 'Arc12'

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [27]:
for ind in range(0,len(years)):

    # ==============================================================================
    # Settings for netcdf file

    save_netcdf       = True                                            # Saves the interpolated field in netcdf file
    delete_old_netcdf = True                                            # If a netcdf file with the same name exists it will be deleted
    saving_directory  = '/scratch/usr/hbkoziel/Arc12/netcdf/'                                    # Where the netcdf is saved
    netcdf_name       = runid +'.'+str(years[ind])+'.monthly.ice.mean.nc'    # Name of netcdf with interpolated DIN field
    plot_netcdf       = False                                          # Reads DIN from the created netcdf file, else it plots the interpolated field (should be the same)

    # ==============================================================================
    # Loading data

    ncfile  = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.ice.mean.nc'
    f       = Dataset(ncfile,'r')
    tracer  = f.variables['area'][:]
    t1 = np.mean(tracer[0:15,:],axis=0)
    t2 = np.mean(tracer[15:29,:],axis=0)
    t3 = np.mean(tracer[29:45,:],axis=0)
    t4 = np.mean(tracer[45:60,:],axis=0)
    t5 = np.mean(tracer[60:75,:],axis=0)
    t6 = np.mean(tracer[75:90,:],axis=0)
    t7 = np.mean(tracer[90:106,:],axis=0)
    t8 = np.mean(tracer[106:121,:],axis=0)
    t9 = np.mean(tracer[121:136,:],axis=0)
    t10 = np.mean(tracer[136:152,:],axis=0)
    t11 = np.mean(tracer[152:167,:],axis=0)
    t12 = np.mean(tracer[166:,:],axis=0)
    tracer = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12]

    tracershape = np.shape(tracer)

    # ==============================================================================
    # Testing if a netcdf file with the same name exists, if yes, it must be removed
    # to save a new one.

    if os.path.isfile(saving_directory+netcdf_name) and delete_old_netcdf:
      os.remove(saving_directory+netcdf_name)
      print "The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name."
    elif os.path.isfile(netcdf_name):
      statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings."
      sys.exit(statement)

    if not os.path.isdir(saving_directory):
      os.makedirs(saving_directory)
      print 'Directory '+saving_directory+' has been created'

    # ==============================================================================
    # Creating netcdf file
    if save_netcdf:
      import time
      w_nc_fid = Dataset(saving_directory+netcdf_name, 'w', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
      w_nc_fid.description = "Sea ice conc"
      w_nc_fid.history     = 'Created ' + time.ctime(time.time())

      nod3d    = w_nc_fid.createDimension('nod2d', mesh.n2d)               # Create dimension: number of 3d nodes
      time     = w_nc_fid.createDimension('time', tracershape[0])

      w_nc_var = w_nc_fid.createVariable(tracername, 'f8',('time','nod2d'))           # 'DIN' is name of saved variable
                                                                           # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u"Sea ice conc",\
                          'units': u"0 to 1"})
      w_nc_fid.variables[tracername][:] = tracer
      w_nc_fid.close()                                                     # close the new file                

      cwd = os.getcwd()
      print "New netcdf file (",netcdf_name,") has been created."
      print "Location: "+saving_directory
      del w_nc_fid, time, nod3d
    else:
      print 'You have specified not to save your field in netcdf file'

New netcdf file ( Arc12.1980.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1981.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1982.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1983.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1984.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1985.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1986.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1987.monthly.ice.mean.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
New netcdf file ( Arc12.1988.monthly.ice.mean.nc ) has been created.
Location: /

NameError: name 'time' is not defined

In [ ]:
# ==============================================================================
# Plot the surface of interpolated DIN field

print '***'
if plot_netcdf:
  ncfile     = saving_directory+netcdf_name
  f          = Dataset(ncfile, 'r')
  data1      = f.variables[tracername][8,0:len(mesh.x2)]
  print "Plotting from netcdf file"
#else:
#  data1 = din_int[0:len(mesh.x2)]
#  print "Plotting interpolated phy N field"

contours = (0.,1.,.01)
contours = np.arange(contours[0], contours[1]+contours[2], contours[2])

fig = plt.figure(num=1, figsize=(12, 8), facecolor='w', edgecolor='k')
ax=plt.subplot(111)
elem2=mesh.elem[mesh.no_cyclic_elem,:]
d=data1[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
bmap = Basemap(projection='robin',lon_0=0)
x, y = bmap(mesh.x2, mesh.y2)
bmap.drawmapboundary(fill_color='0.9')
bmap.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
bmap.drawparallels(np.arange(-90,90,30),labels=plabels)
bmap.drawmeridians(np.arange(bmap.lonmin,bmap.lonmax+30,60),labels=mlabels)
im=plt.tricontourf(x, y, elem2, data1, levels=contours, cmap='rainbow', norm=PiecewiseNorm(contours))
label = 'Sea-Ice Con'
plt.title(label)
cbar=bmap.colorbar(im,"bottom", size="5%", pad="2%")
#cbar.set_label(r'[mmol m$^{-3}$]')
cbar.set_label('%')


#plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
#plt.show()